In [1]:
import pandas as pd
import re
import torch
import sys
transcripts_dir = "/Users/akshaykekuda/Desktop/CSR-SA/manual_score_transcriptions/server_transcripts/"

In [2]:
import pandas as pd
import os
def prepare_inference_df(transcripts_dir):
    df = pd.DataFrame()
    for file in os.listdir(transcripts_dir):
        if file.endswith('.txt'):
            file_loc = transcripts_dir + file
            reg_comps = re.split("_|-|\.", file)
            id = reg_comps[1]
            df.loc[id, 'file_name'] = file_loc
    return df

In [4]:
df = prepare_inference_df(transcripts_dir)

In [5]:
from Preprocessing import preprocess_transcript
from torch.utils.data import Dataset

class InferenceCallDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.df['text'] = self.df.file_name.apply(lambda x: preprocess_transcript(x))
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        text = self.df.iloc[idx]['text']
        id = self.df.index[idx]
        sample = {'text': text, 'id': id}
        return sample

In [6]:
dataset_transcripts_inference = InferenceCallDataset(df)

In [7]:
import pickle
f = open('/Users/akshaykekuda/Desktop/CSR-SA/attention_model/vocab', 'rb')
vocab = pickle.load(f)

In [8]:
sys.path.append("/Users/akshaykekuda/Desktop/CSR-SA/attention_model")
from DataLoader_fns import Collate
from torch.utils.data import DataLoader 
c = Collate(vocab, "cpu")
batch_size = 1
num_workers = 0
dataloader_infer_calls = DataLoader(dataset_transcripts_inference, batch_size=batch_size, shuffle=False,
                                                 num_workers=num_workers, collate_fn=c.collate)

In [10]:
base_dir = "/Users/akshaykekuda/Desktop/CSR-SA/server_logs/cross_selling_sales_cls_01_good1/"
trained_model = torch.load(base_dir+"call_score.model")

In [12]:
base_dir = "/Users/akshaykekuda/Desktop/CSR-SA/server_logs/cross_selling_sales_cls_01_good1/"
trained_model = torch.load(base_dir+"call_score.model")

In [15]:
trained_model.eval()
with torch.no_grad():
    for batch in dataloader_infer_calls:
        if batch['id'][0] == "200148059040210116":
            out, score, value = trained_model(batch['indices'], batch['lens'], batch['trans_pos_indices'],
                                            batch['word_pos_indices'])
            id = batch['id'][0]
            print("id = {}".format(id))

id = 200148059040210116


In [16]:
trained_model.encoder.word_self_attention

WordSelfAttention(
  (embedding): Embedding(17296, 300, padding_idx=1)
  (multihead_attn): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=300, out_features=300, bias=True)
    )
    (linear1): Linear(in_features=300, out_features=600, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear2): Linear(in_features=600, out_features=300, bias=True)
    (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
  )
  (ffn): Linear(in_features=300, out_features=512, bias=True)
  (position_encoding): Embedding(128, 300, padding_idx=0)
)

In [23]:
Wq = trained_model.state_dict()['encoder.sentence_self_attention.multihead_attn.self_attn.in_proj_weight'][-512:,]

In [24]:
trained_model.state_dict()['encoder.sentence_self_attention.multihead_attn.self_attn.in_proj_weight'].shape

torch.Size([1536, 512])

In [25]:
val_norms = (value.squeeze()@ Wq).norm(dim=1)

In [33]:
att_vec = scores@(value.squeeze()@ Wq)

In [37]:
(value.squeeze()@ Wq)[0]

tensor(16.8261)

In [38]:
len(df.loc["200148059040210116"].text)

78

In [29]:
pd.set_option('display.max_colwidth', None)
combined_df = pd.DataFrame()
combined_df['Text'] = df.loc["200148059040210116"].text
combined_df['Scores'] = val_norms
combined_df = combined_df[~(combined_df.Text=='<pad>')].sort_values(by='Scores')
low_df = combined_df[:10]
high_df = combined_df[-1:-20:-1]

In [30]:
high_df

,Text,Scores
45,that that that okay to you yeah okay what do you does that come with it,17.014351
32,one g one z for the,17.006948
31,uh,17.002808
28,and there is any accidents or violations that would because of your life to change for your feet,16.958488
48,price difference but like smaller coverages like that are usually like a couple of dollars,16.879229
63,now also um they do go back five years so have you had any accidents or violations in,16.877014
43,now what type of coverage are you looking for full coverage okay what a five hundred dollar deductible be okay,16.876574
62,checked out for you it should only take about two to three minutes that is fun,16.872005
30,pick great now moving on to vehicles what is the vin number to the vehicle you like to insure,16.870077
19,uh,16.845230


In [31]:
low_df

,Text,Scores
2,i am calling from the get a quote for an insurance policy sure i can help you with the quote what state do you live in,16.407694
44,or one thousand okay would you like thousand roadside service ten rental car,16.456697
47,okay you can add it and i mean it is,16.463619
26,so if there was anything different than that many i am sorry to say it again,16.466892
21,thank you so much and have you had any accidents or violations in the past five years,16.467314
34,h f or frank two six seven zero six seven,16.489016
70,yes i think you do you think,16.511984
69,insurance so let me go ahead and continue with this progressive i am just going to plays piano very pretty pulled settle,16.521936
64,five years no mhm and have you been insured with anybody else within the past,16.528780
60,your information is good for thirty days now pricing can change them just now to,16.529470


In [39]:
head0 = score.squeeze()[0]
head1 = score.squeeze()[1]
head2 = score.squeeze()[2]
head3 = score.squeeze()[3]

In [42]:
head0[0]

tensor([0.0044, 0.0067, 0.0053, 0.0075, 0.0051, 0.0039, 0.0046, 0.0037, 0.0050,
        0.0075, 0.0046, 0.0062, 0.0052, 0.0084, 0.0076, 0.0191, 0.0060, 0.0083,
        0.0130, 0.0066, 0.0048, 0.0048, 0.0095, 0.0082, 0.0106, 0.0056, 0.0051,
        0.0063, 0.0053, 0.0080, 0.0083, 0.0082, 0.0085, 0.0086, 0.0053, 0.0056,
        0.0077, 0.0044, 0.0063, 0.0093, 0.0059, 0.0058, 0.0090, 0.0088, 0.0091,
        0.0072, 0.0051, 0.0093, 0.4553, 0.0032, 0.0085, 0.0053, 0.0114, 0.0062,
        0.0066, 0.0086, 0.0049, 0.0088, 0.0061, 0.0056, 0.0061, 0.0107, 0.0050,
        0.0052, 0.0084, 0.0047, 0.0068, 0.0071, 0.0048, 0.0069, 0.0097, 0.0061,
        0.0108, 0.0080, 0.0058, 0.0107, 0.0057, 0.0077])

In [43]:
pd.set_option('display.max_colwidth', None)
combined_df = pd.DataFrame()
combined_df['Text'] = df.loc["200148059040210116"].text
combined_df['Scores'] = head0[0]
combined_df = combined_df[~(combined_df.Text=='<pad>')].sort_values(by='Scores')
low_df = combined_df[:10]
high_df = combined_df[-1:-20:-1]
high_df

,Text,Scores
48,price difference but like smaller coverages like that are usually like a couple of dollars,0.455253
15,okay great so couple mar questioned what is your marital,0.019147
18,yeah,0.012954
52,if he ever have any issues with like transmission breaks attention but that be something like tickets,0.011420
72,okay thank you for holding so you know what actually progressive is right around the same amount but they require more they want to fifty two to start to us,0.010770
75,which is a company called insurance america so we do have representatives that can help you get a quote with them would you want to do that,0.010713
61,on like when you call into started now since he are shopping around what i can do is i can also provide you will quote winter grass it heavy guy in one with them already know i get,0.010711
24,yeah okay and um do you need an s r twenty two,0.010634
70,yes i think you do you think,0.009661
22,no,0.009484


In [44]:
low_df

,Text,Scores
49,a month for that,0.003201
7,if i can also get your full social please to eight eight,0.003711
5,why thank you and have you ever had safe auto before yeah okay,0.003852
37,one g one z for zebra you for elephant,0.004392
10,six three nine gary g a r r y road akron ohio for four three zero five,0.004552
6,what is it your own policy or were you under somebody else my own policy okay,0.004562
65,thirty days yes what company are you currently with or have you been with,0.004658
21,thank you so much and have you had any accidents or violations in the past five years,0.004769
20,new q two five three to five seven,0.004812
68,okay perfect so you are getting a discount for you know having continuing,0.004829


In [45]:
pd.set_option('display.max_colwidth', None)
combined_df = pd.DataFrame()
combined_df['Text'] = df.loc["200148059040210116"].text
combined_df['Scores'] = head1[0]
combined_df = combined_df[~(combined_df.Text=='<pad>')].sort_values(by='Scores')
low_df = combined_df[:10]
high_df = combined_df[-1:-20:-1]
high_df

,Text,Scores
40,eight for injury f for frank two six seven zero six four,0.057907
48,price difference but like smaller coverages like that are usually like a couple of dollars,0.023637
44,or one thousand okay would you like thousand roadside service ten rental car,0.022711
23,hum occupation was previously lifted ask customer service so that still the,0.020912
14,an email violet as in the color power one one five at gmail dot com,0.018469
76,i have to ask for salmon d for dog no i am fine thank you well if anything where here twenty four seven,0.016948
36,one g as in golf one z as in zebra for the s in zebra hill so it is ah,0.016911
47,okay you can add it and i mean it is,0.016686
52,if he ever have any issues with like transmission breaks attention but that be something like tickets,0.016421
21,thank you so much and have you had any accidents or violations in the past five years,0.016276


In [46]:
low_df

,Text,Scores
30,pick great now moving on to vehicles what is the vin number to the vehicle you like to insure,0.006520
66,well i see and how long have you been with them,0.007303
41,have two thousand seventeen chevrolet malibu yeah,0.007388
43,now what type of coverage are you looking for full coverage okay what a five hundred dollar deductible be okay,0.008097
20,new q two five three to five seven,0.008158
57,and then this is what the thousand dollar deductible,0.008263
64,five years no mhm and have you been insured with anybody else within the past,0.008301
31,uh,0.008313
45,that that that okay to you yeah okay what do you does that come with it,0.008447
19,uh,0.008548


In [47]:
pd.set_option('display.max_colwidth', None)
combined_df = pd.DataFrame()
combined_df['Text'] = df.loc["200148059040210116"].text
combined_df['Scores'] = head2[0]
combined_df = combined_df[~(combined_df.Text=='<pad>')].sort_values(by='Scores')
low_df = combined_df[:10]
high_df = combined_df[-1:-20:-1]
high_df

,Text,Scores
57,and then this is what the thousand dollar deductible,0.049481
72,okay thank you for holding so you know what actually progressive is right around the same amount but they require more they want to fifty two to start to us,0.024460
51,with a year of your vehicle you do qualify to get a mechanical service plan so that would cover mechanical breakdown,0.023070
61,on like when you call into started now since he are shopping around what i can do is i can also provide you will quote winter grass it heavy guy in one with them already know i get,0.019876
17,and then i do have your data birth as eleven five ninety nine yeah can can i also get your drivers license number please,0.018152
48,price difference but like smaller coverages like that are usually like a couple of dollars,0.017864
10,six three nine gary g a r r y road akron ohio for four three zero five,0.016954
20,new q two five three to five seven,0.016910
71,okay hello madam i am still here,0.016529
68,okay perfect so you are getting a discount for you know having continuing,0.016503


In [48]:
pd.set_option('display.max_colwidth', None)
combined_df = pd.DataFrame()
combined_df['Text'] = df.loc["200148059040210116"].text
combined_df['Scores'] = head3[0]
combined_df = combined_df[~(combined_df.Text=='<pad>')].sort_values(by='Scores')
low_df = combined_df[:10]
high_df = combined_df[-1:-20:-1]
high_df

,Text,Scores
49,a month for that,0.025769
17,and then i do have your data birth as eleven five ninety nine yeah can can i also get your drivers license number please,0.017748
14,an email violet as in the color power one one five at gmail dot com,0.017428
54,that is something you can always add in the future so right now i am going to quote you with er rental roadside what did you also like to include any type of like,0.017119
35,i might have wrote down something wrong someone what i have just got to run you to correct it so i have,0.016659
39,t protest on one,0.016356
12,you be interested in renters insurance nerdy have the okay,0.016016
18,yeah,0.015838
63,now also um they do go back five years so have you had any accidents or violations in,0.015789
68,okay perfect so you are getting a discount for you know having continuing,0.015617


In [ ]:
low_df

In [ ]:
encoder_layer = torch.nn.TransformerEncoderLayer(d_model=300, nhead=6)
src = torch.rand(10, 32, 300)
out = encoder_layer(src)

In [ ]:
encoder_layer = torch.nn.TransformerEncoderLayer(d_model=300, nhead=4, dropout=0.5, batch_first=True, dim_feedforward =64)

In [ ]:
multihead_attn = torch.nn.MultiheadAttention(300, dropout=0.5, num_heads=4, batch_first=True)


In [ ]:
t = torch.tensor([1, float('nan'), 2])

In [ ]:
t.isnan().sum()

In [10]:
import pickle
import pandas as pd

In [14]:
test_df = pd.read_pickle("/Users/akshaykekuda/Desktop/CSR-SA/server_logs/ak_fbck_up3.2_m64_01/call_score_test.p")

In [17]:
len(test_df)

1017